# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [4]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [5]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [6]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [6]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [7]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [8]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

Creating aliases for patent and citation. Also, removing the null states in the patents table.

In [9]:
p = patents.alias('p1')
c = citations.alias('c')
p = p.select(col('*')).filter(col('POSTATE') != '')

Creating the first join where we match the patents of citing in citation table to patent in patents table

In [10]:
join1 = c.join(p, c.CITING == p.PATENT)\
         .select(c.CITING.alias("Citing"),p.POSTATE.alias("Citing_State"),c.CITED.alias("Cited"))

In [11]:
join1.show(5)

+-------+------------+-------+
| Citing|Citing_State|  Cited|
+-------+------------+-------+
|3858242|          MI|1515701|
|3858242|          MI|3319261|
|3858242|          MI|3668705|
|3858242|          MI|3707004|
|3858244|          CT|  14040|
+-------+------------+-------+
only showing top 5 rows



Taking the previous join to create a new join to match the patent of cited in citations to patent in patents table generated from previous query. Also, displaying the intermediate result as shown.

In [21]:
join2 = join1.join(p, join1.Cited == p.PATENT)\
             .select("Cited",col("POSTATE").alias("Cited_State"),"Citing","Citing_State").orderBy(col("Cited"))

In [22]:
join2.show(10)

+-------+-----------+-------+------------+
|  Cited|Cited_State| Citing|Citing_State|
+-------+-----------+-------+------------+
|3070803|         IL|4483021|          MS|
|3070803|         IL|5557807|          FL|
|3070803|         IL|4921141|          CA|
|3070803|         IL|4484363|          CA|
|3070803|         IL|4133055|          NH|
|3070803|         IL|5850636|          CA|
|3070804|         OH|4694843|          OH|
|3070804|         OH|5450626|          TX|
|3070804|         OH|3972325|          CA|
|3070805|         CA|4400830|          FL|
+-------+-----------+-------+------------+
only showing top 10 rows



 Filtering with the same state and performing groupBy and taking the aggregate of the count in decreasing order

In [19]:
filter1 = join2.filter(col("Cited_State") == col("Citing_State"))\
               .groupBy("Citing","Citing_State").agg(count("*").alias("SAME_STATE"))\
               .orderBy(col("SAME_STATE").desc())

In [20]:
filter1.show(10)

+-------+------------+----------+
| Citing|Citing_State|SAME_STATE|
+-------+------------+----------+
|5959466|          CA|       125|
|5983822|          TX|       103|
|6008204|          CA|       100|
|5952345|          CA|        98|
|5958954|          CA|        96|
|5998655|          CA|        96|
|5936426|          CA|        94|
|5951547|          CA|        90|
|5980517|          CA|        90|
|5739256|          CA|        90|
+-------+------------+----------+
only showing top 10 rows



Displaying the final output with the required columns

In [90]:
finalAns = p.join(filter1, p.PATENT == filter1.Citing)\
            .drop(col("Citing")).drop(col("Citing_State"))\
            .orderBy(col("SAME_STATE").desc())
finalAns.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|       103|
|6008204| 1999|14606|   1998| 